<a href="https://colab.research.google.com/github/jellothere/CNN-vs-MLP/blob/main/multilayer-perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pactools

     |████████████████████████████████| 82 kB 987 kB/s 
     |████████████████████████████████| 7.4 MB 17.0 MB/s 


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report
from keras.wrappers.scikit_learn import KerasRegressor
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
# Importación de los archivos desde Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_set = pd.read_csv('/content/drive/MyDrive/P2_RNA/vehicleTrainValid_0-1.csv', header=None, delimiter=',')
test_set = pd.read_csv('/content/drive/MyDrive/P2_RNA/vehicleTest_0-1.csv', header=None, delimiter=',')

# SELECCION DE LA SALIDA
y_train = train_set.iloc[:,-1:]
X_train = train_set.iloc[: , :-1]
y_test = test_set.iloc[:,-1:]
X_test = test_set.iloc[: , :-1]

In [ ]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.086957,-0.538462,0.138889,-0.240175,-0.626374,-0.735849,-0.294118,-0.200000,-0.500000,-0.428571,-0.431579,-0.446043,-0.723270,-0.921053,-0.272727,-0.073171,0.400000,0.600000
1,-0.304348,-0.153846,0.000000,-0.816594,-0.846154,-0.698113,-0.424837,0.028571,-0.500000,-0.057143,-0.652632,-0.597122,-0.157233,-0.552632,0.090909,-0.414634,-0.466667,-0.200000
2,-0.434783,-0.230769,-0.222222,-0.598253,-0.626374,-0.735849,-0.660131,0.257143,-0.833333,-0.285714,-0.673684,-0.757794,-0.182390,-0.736842,-0.454545,-1.000000,0.066667,0.333333
3,-0.478261,-0.769231,-0.111111,-0.799127,-0.802198,-0.811321,-0.803922,0.600000,-0.833333,-0.800000,-0.852632,-0.882494,-0.823899,-0.710526,-0.727273,0.463415,-0.466667,-0.133333
4,0.173913,0.615385,0.722222,-0.109170,-0.604396,-0.698113,0.124183,-0.600000,0.000000,0.314286,-0.010526,-0.033573,0.647799,-0.657895,-0.636364,-0.463415,-0.266667,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,-0.521739,-0.153846,0.111111,-0.685590,-0.758242,-0.660377,-0.424837,0.028571,-0.500000,0.114286,-0.621053,-0.604317,-0.157233,-0.605263,-0.545455,-0.170732,-0.533333,-0.200000
560,-0.434783,-0.153846,-0.305556,-0.781659,-0.802198,-0.849057,-0.477124,0.085714,-0.666667,-0.085714,-0.600000,-0.647482,-0.144654,-0.368421,-0.636364,-0.365854,-0.866667,-0.866667
561,0.086957,-0.076923,0.000000,-0.458515,-0.714286,-0.811321,-0.411765,-0.085714,-0.500000,-0.142857,-0.431579,-0.551559,-0.031447,-0.736842,0.181818,-0.512195,-0.066667,0.000000
562,0.434783,0.538462,0.611111,-0.222707,-0.714286,-0.698113,0.346405,-0.714286,0.166667,0.457143,-0.010526,0.191847,0.371069,-0.631579,-0.272727,0.024390,-0.200000,0.066667


In [ ]:
# CONVERTIR TARGET A CATEGORICAL
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
y_train_transformed = encoder.fit_transform(y_train)
y_test_transformed = encoder.fit_transform(y_test)

print(y_train[:11])
print(y_train_transformed[:11])

    saab
0    van
1    van
2   opel
3   saab
4    bus
5   saab
6   opel
7   opel
8    bus
9    bus
10   bus
[[0 0 0 1]
 [0 0 0 1]
 [0 1 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 1 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]]


In [ ]:
# COMPROBAR DIMENSIONES DE LOS DATOS
print(X_train.shape)
print(y_train.shape)
print(y_train_transformed.shape)
print(X_test.shape)
print(y_test.shape)
print(y_test_transformed.shape)

#obtener dim de la entrada y número de salidas
input_shape = (X_train.shape [1] ,) # (18, )
num_clases = y_train_transformed.shape [1] # 4

(563, 18)
(563, 1)
(563, 4)
(281, 18)
(281, 1)
(281, 4)


In [ ]:
from itertools import product
from tensorflow.keras import initializers

#Definición del modelo
def genera_modelo(arch, lr, m):
  model = Sequential()
  for i in arch:
    model.add(Dense(i, input_shape=input_shape, activation='sigmoid'))
  model.add(Dense(4, activation='softmax'))
  model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.SGD(learning_rate=lr, momentum=m), metrics=['accuracy','mse'])
  return model

def generarCapasOcultas():
    res = list()
    for i in range(2,3):
        res += list(product(range(400, 700+1, 100), repeat = i))
    return res

In [ ]:
# Creación del modelo
model = KerasClassifier(build_fn=genera_modelo, verbose=0)

# Parámetros para GridSearch
architecture = generarCapasOcultas()
learning_rate = [0.2]
momentum = [0]
epochs = [500]

# Definición del GridSearchCV
param_grid = dict(arch=architecture, lr = learning_rate, m=momentum, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3, verbose=5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  


In [ ]:
grid_result = grid.fit(X_train, y_train_transformed, batch_size=1, shuffle=False, verbose=1)

Fitting 3 folds for each of 25 candidates, totalling 75 fits
Epoch 1/500
375/375 [==============================] - 2s 3ms/step - loss: 0.2342 - accuracy: 0.2400 - mse: 0.2342
Epoch 2/500
375/375 [==============================] - 1s 3ms/step - loss: 0.2070 - accuracy: 0.2613 - mse: 0.2070
Epoch 3/500
375/375 [==============================] - 1s 3ms/step - loss: 0.1959 - accuracy: 0.2720 - mse: 0.1959
Epoch 4/500
375/375 [==============================] - 1s 3ms/step - loss: 0.1912 - accuracy: 0.2773 - mse: 0.1912
Epoch 5/500
375/375 [==============================] - 1s 3ms/step - loss: 0.1883 - accuracy: 0.2800 - mse: 0.1883
Epoch 6/500
375/375 [==============================] - 1s 3ms/step - loss: 0.1860 - accuracy: 0.2987 - mse: 0.1860
Epoch 7/500
375/375 [==============================] - 1s 3ms/step - loss: 0.1839 - accuracy: 0.3227 - mse: 0.1839
Epoch 8/500
375/375 [==============================] - 1s 3ms/step - loss: 0.1820 - accuracy: 0.3253 - mse: 0.1820
Epoch 9/500
375/375

# Resultados de GridSearchCV
Se almacenan en un fichero para poder ser evaluados con posterioridad

In [ ]:
# Se obtienen los elementos de evaluación deseados del GridSearch
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
ranking = grid_result.cv_results_['rank_test_score']
params = grid_result.cv_results_['params']

In [ ]:
# Almacenando los resultados obtenidos de GridSearch
resultados = []
for i in range(len(ranking)):
  # Convertimos la tupla de la arquitectura en su representación equivalente como cadena
  arch = '(' +  ', '.join(str(e) for e in list(params[i]['arch'])) + ')'
  experimento = [ranking[i],means[i],stds[i],arch,params[i]['epochs'],params[i]['lr'],params[i]['m']]
  resultados.append(experimento)

In [ ]:
# Convirtiendo la lista de listas en dataframe de pandas para mayor claridad
resultados = pd.DataFrame(resultados, columns=['ranking','mean','std','arch','epochs','lr','momentum'])

In [ ]:
resultados

,ranking,mean,std,arch,epochs,lr,momentum
0,2,0.801077,0.026450,(300),500,0.2,0
1,3,0.793956,0.028286,(400),500,0.2,0
2,1,0.802822,0.023158,(500),500,0.2,0
3,4,0.790477,0.051220,(600),500,0.2,0
4,5,0.763824,0.040055,(700),500,0.2,0


In [ ]:
# Guardando los resultados en un fichero CSV
resultados.to_csv("/content/drive/MyDrive/P2_RNA/rna_p2_gs_2layers.csv", index=False)